# Day 4: Scratchcards

Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).

As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.

## Part 1

In [24]:
module Data =
    let load fileName =
        File.ReadLines fileName

module String = 
    open System.Text.RegularExpressions

    let split (token: string) (input: string) = (Regex token).Split(input)
        
    let trim (input: string) = input.Trim()

module Combinators =
    let inline C a b c = a c b
    let inline K a _ = a
    let inline Φ a b c d = a (b d) (c d)
    
module Math =
    let max (a: int) (b: int) =
        Math.Max(a, b)
    
    let pow x y =
        Math.Pow(x, y)
        
module Tuple =
    let make a b =
        (a, b)

type Card =
    { Id: int
      Winning: int seq 
      Guesses: int seq }
    
type WinningCard =
    { Id: int
      Matches: int seq }
    
type Rewards =
    { Id: int
      Copies: int seq }
    
module Card =
    open System.Linq

    let private intersect (xs: 'a seq) (ys: 'a seq) = xs.Intersect ys
    
    let private makeWinningCard id matches =
        { Id = id; Matches = matches }

    let findWinning (card: Card) =
        makeWinningCard card.Id (intersect card.Winning card.Guesses)
        
    let rewardCopies (card: WinningCard) =
        { Id = card.Id; Copies = Seq.init (Seq.length card.Matches) ((+) (card.Id + 1)) }

    let getMatches (card: WinningCard) =
        card.Matches

    let score (xs: 'a seq) =
        if Seq.isEmpty xs then
            0.0
        else
            (Seq.length xs - 1)
            |> Combinators.C Math.max 0
            |> float
            |> Math.pow 2
            
    let private makeCard id (winning, guesses) =
        { Id = id; Winning = winning; Guesses = guesses }
        
    let private addCards number id cards =
        let count = cards |> Map.tryFind id |> Option.defaultValue 0
        cards |> Map.add id (count + number)
        
    let countCards cards (rewards: Rewards) =
        let added = cards |> addCards 1 rewards.Id
        let count = Map.find rewards.Id added
        Seq.fold (Combinators.C (addCards count)) added rewards.Copies

    let parseId =
        String.split ":"
        >> Seq.head
        >> String.split " "
        >> Seq.last
        >> int

    let parseWinning =
        String.split ":"
        >> Seq.last
        >> String.trim
        >> String.split "\|"
        >> Seq.map (String.trim >> String.split "\s+")
        >> Seq.map (Seq.map int)
        >> Combinators.Φ Tuple.make Seq.head Seq.last
        
    let parse =
        Combinators.Φ makeCard parseId parseWinning

### Test

In [108]:
"Day 4 - Part 1 - Test.txt"
|> Data.load
|> Seq.map (Card.parse >> Card.findWinning >> Card.getMatches >> Card.score)
|> Seq.sum

13

### Solution

In [109]:
"Day 4 - Part 1.txt"
|> Data.load
|> Seq.map (Card.parse >> Card.findWinning >> Card.getMatches >> Card.score)
|> Seq.sum

23441

## Part 2

### Test

In [18]:
"Day 4 - Part 1 - Test.txt"
|> Data.load
|> Seq.map (Card.parse >> Card.findWinning >> Card.rewardCopies)
|> Seq.fold Card.countCards Map.empty
|> Map.values |> Seq.sum

30

### Solution

In [25]:
open System.Collections.Generic

"Day 4 - Part 2.txt"
|> Data.load
|> Seq.map (Card.parse >> Card.findWinning >> Card.rewardCopies)
|> Seq.fold Card.countCards Map.empty
|> Map.values |> Seq.sum

5923918